In [1]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Read in data from CSV files using pandas read_csv 
labs = pd.read_csv('labs.csv')
exam = pd.read_csv('examination.csv')
demo = pd.read_csv('demographic.csv')
diet = pd.read_csv('diet.csv')
ques = pd.read_csv('questionnaire.csv')

#Remove the SEQN column from examination, demographic, diet, and questionnaire dataframes
exam.drop(['SEQN'], axis = 1, inplace=True)
demo.drop(['SEQN'], axis = 1, inplace=True)
diet.drop(['SEQN'], axis = 1, inplace=True)
ques.drop(['SEQN'], axis = 1, inplace=True)

#Concatenate the dataframes along columns (axis=1) using inner join and store the resulting dataframe in 'df'
df = pd.concat([labs, exam], axis=1, join='inner')
df = pd.concat([df, demo], axis=1, join='inner')
df = pd.concat([df, diet], axis=1, join='inner')
df = pd.concat([df, ques], axis=1, join='inner')

In [2]:
d= df

In [3]:
#Data standardization
d.replace({7:None, 9:None, 77:None,99:None,777:None,999:None,7777:None,9999:None,77777:None,99999:None,
            777777:None,999999:None,55:None,555:None,5555:None,8:None,88:None}, inplace=True)

In [4]:
#Removing highly specific columns, HPV type, specific tooth missing, sample weights, metadata
drop_cols = [ 
             'BPQ080', 'LBDBANO','OHX01TC', 'OHX02TC', 'OHX03TC', 'OHX04TC', 'OHX05TC', 'OHX06TC', 'OHX07TC', 'OHX08TC', 'OHX09TC', 
             'OHX10TC', 'OHX11TC', 'OHX12TC', 'OHX13TC', 'OHX14TC', 'OHX15TC', 'OHX16TC', 'OHX17TC', 'OHX18TC', 'OHX19TC', 'OHX20TC', 'OHX21TC', 'OHX22TC', 'OHX23TC', 'OHX24TC', 
             'OHX25TC', 'OHX26TC', 'OHX27TC', 'OHX28TC', 'OHX29TC', 'OHX30TC', 'OHX31TC', 'OHX32TC', 'OHX02CTC', 'OHX03CTC', 'OHX04CTC', 'OHX05CTC', 'OHX06CTC', 'OHX07CTC', 
             'OHX08CTC', 'OHX09CTC', 'OHX10CTC', 'OHX11CTC', 'OHX12CTC', 'OHX13CTC', 'OHX14CTC', 'OHX15CTC', 'OHX18CTC', 'OHX19CTC', 'OHX20CTC', 'OHX21CTC', 'OHX22CTC', 
             'OHX23CTC', 'OHX24CTC', 'OHX25CTC', 'OHX26CTC', 'OHX27CTC', 'OHX28CTC', 'OHX29CTC', 'OHX30CTC', 'OHX31CTC','DMDHRAGE','WTINT2YR','WTMEC2YR','SEQN','RIDSTATR'
            ]
d.drop(drop_cols, axis=1,inplace=True)

In [5]:
#Move to Weka for quick remove of string and non-vary attributes
#d.to_csv(r'1.csv', index=False)

In [6]:
#Weka file back to jupyternotebook
df = pd.read_csv('weka.csv')

In [7]:
#Changing RIAGENDR back to 1(male), 2(female) 
df['RIAGENDR']=df.apply(lambda x: 1 if x.RIAGENDR == 'male' else 2, axis='columns')
df.RIAGENDR.value_counts()

2    2910
1    2647
Name: RIAGENDR, dtype: int64

In [8]:
new_df=df
# less than 40% non-null values will be dropped
new_df= new_df.dropna(thresh = 0.4*len(new_df), axis =1)

print(len(new_df.columns), 'columns left')

642 columns left


In [9]:
#Fillin with most_frequent
from sklearn.impute import SimpleImputer
imp_mode=SimpleImputer(strategy='most_frequent')
df_mostfre = pd.DataFrame(imp_mode.fit_transform(new_df), columns=new_df.columns)


In [10]:
#Move stroke target class to the last
cols = df_mostfre.columns.tolist()
cols.append(cols.pop(cols.index('MCQ160F')))
df_mostfre = df_mostfre[cols]

In [11]:
#Export df for stroke model
df_mostfre.to_csv(r'df_mostfre_stroke_.csv', index=False)

In [12]:
#Move diabetes target class to the last
cols = df_mostfre.columns.tolist()
cols.append(cols.pop(cols.index('DIQ010')))
df_mostfre = df_mostfre[cols]

In [13]:
#Export df for diabetes model
df_mostfre.to_csv(r'df_mostfre_diabetes_.csv', index=False)